In [5]:
%load_ext autoreload
%autoreload 2

from time import time
import pandas as pd
import numpy as np
import os
from collections import Counter, defaultdict
import pickle

In [6]:
import sys
sys.path.insert(0, "/data3/muntean/DRhard")

In [7]:
import argparse
import subprocess
import sys
sys.path.append("./")
import faiss
import logging
import os
import numpy as np
# import torch
from transformers import RobertaConfig
from tqdm import tqdm
from torch.utils.data.dataloader import DataLoader
from torch.utils.data.sampler import SequentialSampler

from model import RobertaDot
from dataset import (
    TextTokenIdsCache, load_rel, SubsetSeqDataset, SequenceDataset,
    single_get_collate_function
)
from retrieve_utils import (
    construct_flatindex_from_embeddings, 
    index_retrieve, convert_index_to_gpu
)
logger = logging.Logger(__name__)

/data3/muntean/DRhard/DRhard/lib/python3.8/site-packages/torch-1.7.0-py3.8-linux-x86_64.egg/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 803: system has unsupported display driver / cuda driver combination (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
doc_memmap_path = "/data3/muntean/DRhard/data/passage/evaluate/star/passages.memmap"
docid_memmap_path = "/data3/muntean/DRhard/data/passage/evaluate/star/passages-id.memmap"
query_memmap_path = "/data3/muntean/DRhard/data/passage/evaluate/star/test-manual-query.memmap"
queryids_memmap_path = "/data3/muntean/DRhard/data/passage/evaluate/star/test-manual-query-id.memmap"

query_adore_memmap_path = "/data3/muntean/DRhard/data/passage/evaluate/adore-star/test-manual.qembed.memmap"

'/data3/muntean/DRhard/data/passage/evaluate/adore-star/test-manual.qembed.memmap'

In [5]:
doc_embeddings = np.memmap(doc_memmap_path, dtype=np.float32, mode="r")
doc_ids = np.memmap(docid_memmap_path, dtype=np.int32, mode="r")
doc_embeddings = doc_embeddings.reshape(-1, 768)

query_embeddings = np.memmap(query_adore_memmap_path, dtype=np.float32, mode="r")
query_embeddings = query_embeddings.reshape(-1, 768)
query_ids = np.memmap(queryids_memmap_path, dtype=np.int32, mode="r")

In [8]:
# query_adore_embeddings = np.memmap(query_adore_memmap_path, dtype=np.float32, mode="r")
# query_adore_embeddings = query_adore_embeddings.reshape(-1, 768)

In [10]:
# len(query_adore_embeddings[1])

768

In [6]:
%time
index = construct_flatindex_from_embeddings(doc_embeddings, doc_ids)

CPU times: user 4 µs, sys: 3 µs, total: 7 µs
Wall time: 12.9 µs
embedding shape: (38626614, 768)
(38626614,) int64


In [7]:
type(index)

faiss.swigfaiss.IndexIDMap2

# Select certain queries and certain docs for small index

In [8]:
# Load our qid and docid remapping dictionaries

# query id dict
qid_mapping_path = "/data3/muntean/DRhard/data/passage/dataset/queries.CASTmanual.QID2newID.test.tsv"
queries_df = pd.read_csv(qid_mapping_path, delimiter="\t", header=None)
print(len(queries_df))

# collection id dict
collection_mapping_path = "/data3/muntean/DRhard/data/passage/dataset/CASTcollectionPID2newID.tsv"
collection_df = pd.read_csv(collection_mapping_path, delimiter="\t", header=None)
print(len(collection_df))

479
38626614


In [9]:
qid2newqid_dict = dict(zip(queries_df[0], queries_df[1])) 
pid2newpid_dict = dict(zip(collection_df[0], collection_df[1])) 

In [10]:
qid2newqid_dict["32_1"]

9

In [11]:
# Create reverse dictionaries
newqid2qid_dict = dict(zip(queries_df[1], queries_df[0])) 
newpid2pid_dict = dict(zip(collection_df[1], collection_df[0])) 

In [12]:
newqid2qid_dict[9]

'32_1'

In [13]:
# DRhard docid and qid encoding
preprocess_dir = "/data3/muntean/DRhard/data/passage/preprocess"

pid2offset = pickle.load(open(os.path.join(preprocess_dir, "pid2offset.pickle"), 'rb'))
offset2pid = {v:k for k, v in pid2offset.items()}
qid2offset = pickle.load(open(os.path.join(preprocess_dir, f"test-manual-qid2offset.pickle"), 'rb'))
offset2qid = {v:k for k, v in qid2offset.items()}

In [14]:
qid2offset[9]

9

# Create conv cache

In [54]:
topk = 10000 # cache dimension [1000,2000,5000,10000]
# evaluation is done on top 1000 - another topk?

In [55]:
# distance dicts

cache_radius_dict = dict() # between first utterance (qi) and last retrieved doc from the big index
query_distance_dict = dict() # distance between the first (qi) and the rest of utterances of the conversation (q)
query_radius_dict = dict() # between current utterance (q) and last retrieved doc from the small index
diff_distance_dict = dict() # rbcapuccio = rqi - d(q, qi)

In [56]:
def l2_distance(v1,v2):
    return np.linalg.norm(v1-v2)

In [57]:
def create_conv_cache(conv_id, qid2newqid_dict, qid2offset, query_embeddings, index, topk, cache_radius_dict):
    first_qid = conv_id + "_1"
    newqid = qid2newqid_dict[first_qid] #added first
    qid_offset = qid2offset[newqid]

    # prendere il memmap
    query_emb = query_embeddings[qid_offset].reshape(1, 768)
    
    # fare retireval nel indice grande e prendere top 2000 documenti
    faiss.omp_set_num_threads(16) #32
    nearest_neighbors = index_retrieve(index, query_emb, topk, batch=32)
    
    
    # select doc embeddings, paired with ids
    small_doc_emb = doc_embeddings[nearest_neighbors[0]]
    small_doc_ids = np.array(nearest_neighbors[0])
    index_conv = construct_flatindex_from_embeddings(small_doc_emb, small_doc_ids)
       
    # compute distance between the first query and last doc in the list of topk retrieved that are stored in cache (e.g., r_q_i)
    last_doc = nearest_neighbors[0][-1]
    cache_radius_dict[first_qid] = l2_distance(query_emb, last_doc)
    

    return index_conv, nearest_neighbors, cache_radius_dict

In [58]:
conv_ids = set([x.split("_")[0] for x in qid2newqid_dict.keys()]) # this has all but we don't need all, just the ones in qrel
# conv in qrel: subset of all conv
conv_qrel_int = [31, 32, 33, 34, 37, 40, 49, 50, 54, 56, 58, 59, 61, 67, 68, 69, 75, 77, 78, 79]
conv_qrel = [str(x) for x in conv_qrel_int]


results_list = []
for conv_id in conv_qrel: # conv_ids:
    
    print("Starting conv: " , conv_id)
    # Create index for first query and retrieve nearest neighbours - top 2000
    index_conv, nearest_neighbors, cache_radius_dict = create_conv_cache(conv_id, qid2newqid_dict, qid2offset, query_embeddings, index, topk, cache_radius_dict)
    
    # save results - top 1000 for first conv query
    newqid = qid2newqid_dict[conv_id + "_1"]
    qid_offset = qid2offset[newqid]
    for idx, pid in enumerate(nearest_neighbors[0][:1000]):
        results_list.append((qid_offset, pid, idx+1))
        
    # first  query id & embedding
    first_qid = conv_id + "_1"
    first_newqid = qid2newqid_dict[first_qid] #added first
    first_qid_offset = qid2offset[first_newqid]
    # prendere il memmap
    first_query_emb = query_embeddings[first_qid_offset].reshape(1, 768)
    
    for qid in qid2newqid_dict.keys():
        if not qid.endswith("_1") and qid.startswith(conv_id):
            # select query embedding
            newqid = qid2newqid_dict[qid]
            qid_offset = qid2offset[newqid]
            # prendere il memmap
            query_emb = query_embeddings[qid_offset].reshape(1, 768)
            # retrieve docs
            faiss.omp_set_num_threads(16) #32
            nearest_neighbors = index_retrieve(index_conv, query_emb, 1000, batch=32)
            # save results
            for idx, pid in enumerate(nearest_neighbors[0]):
                results_list.append((qid_offset, pid, idx+1))
                
            # compute distance between the current query (q) and the first query (qi) (e.g., d(q, qi))
            query_distance_dict[qid] = l2_distance(query_emb, first_query_emb)
            
            #compute distance between current query (q) and the last retrieved doc (from cache ???) (e.g., sq or rb?)
            last_doc = nearest_neighbors[0][-1]
            query_radius_dict[qid] = l2_distance(query_emb, last_doc)
            
            #compute sq = rqi - d(q, qi)
            diff_distance_dict[qid] = cache_radius_dict[first_qid] - query_distance_dict[qid]

Starting conv:  31
Query Num 1


100%|█████████████████████████████████████████████████████| 1/1 [00:45<00:00, 45.83s/it]


Elapsed Time: 45.8s, Elapsed Time per query: 45832.5ms
embedding shape: (10000, 768)
(10000,) int64
Query Num 1


100%|████████████████████████████████████████████████████| 1/1 [00:00<00:00, 103.15it/s]


Elapsed Time: 0.0s, Elapsed Time per query: 13.4ms
Query Num 1


...

Elapsed Time: 0.0s, Elapsed Time per query: 11.5ms
Query Num 1


100%|████████████████████████████████████████████████████| 1/1 [00:00<00:00, 116.13it/s]


Elapsed Time: 0.0s, Elapsed Time per query: 11.5ms
Query Num 1


100%|████████████████████████████████████████████████████| 1/1 [00:00<00:00, 117.24it/s]

Elapsed Time: 0.0s, Elapsed Time per query: 11.3ms


In [59]:
len(results_list)

194000

In [60]:
# convert ids to original
with open("/data3/muntean/conversational-cache/data/star-ranking/CAST-manual-queries-adore-star-L2-ranking-top1000-cache-top"+str(topk)+"-first-utt_new.tsv", 'w') as outputfile:
    for (qid, pid, idx) in results_list:
        
        new_qid = offset2qid[qid]
        orig_qid = newqid2qid_dict[new_qid]
        
        new_pid = offset2pid[pid]
        orig_pid = newpid2pid_dict[new_pid]
        
        outputfile.write(f"{orig_qid}\t{orig_pid}\t{idx}\n")

# Eval results

In [61]:
# import pyterrier as pt
# pt.init()

In [62]:
qrel_path = "../data/CAST_qrels/qrels-docs.2019.txt"
qrels_df = pd.read_csv(qrel_path, delimiter=" ", header=None)
qrels_df[[3]] = qrels_df[[3]].astype(int)
qrels_df = qrels_df.drop([1], axis=1)
qrels_df.columns=["qid", "docno", "label"]
qrels = qrels_df

In [63]:
topics_path='../data/CAST-2019/test_manual_utterance.tsv' #manual

topics_df = pd.read_csv(topics_path, delimiter="\t", header=None)
topics_df.columns=["qid", "query"]
topics = topics_df
topics.head()

,qid,query
0,31_1,What is throat cancer?
1,31_2,Is throat cancer treatable?
2,31_3,Tell me about lung cancer.
3,31_4,What are lung cancer's symptoms?
4,31_5,Can lung cancer spread to the throat?


In [64]:
results_path = "../data/star-ranking/CAST-manual-queries-adore-star-L2-ranking-top1000-cache-top"+str(topk)+"-first-utt_new.tsv"
results_df = pd.read_csv(results_path, delimiter="\t", header=None)
results_df[3] = 1000-results_df[2]
results_df.columns=["qid", "docno", "rank", "score"]
results_df.head()
# Results produced by the transformers must have “qid”, “docno”, “score”, “rank” columns.

,qid,docno,rank,score
0,31_1,MARCO_3878347,1,999
1,31_1,MARCO_789620,2,998
2,31_1,MARCO_291003,3,997
3,31_1,MARCO_5625372,4,996
4,31_1,MARCO_2954451,5,995


In [65]:
%%time
pt.Experiment([results_df], topics, qrels, names=["STAR"], 
              eval_metrics=["map", "recip_rank", "recall_200", "P_3", "P_1", "ndcg_cut_3"])

CPU times: user 445 ms, sys: 0 ns, total: 445 ms
Wall time: 444 ms


,name,map,recip_rank,recall_200,P_3,P_1,ndcg_cut_3
0,STAR,0.135279,0.593401,0.329988,0.429672,0.468208,0.324213


In [66]:
%%time
res_per_query = pt.Experiment([results_df], topics, qrels, names=["STAR"], 
              eval_metrics=["map", "recip_rank", "recall_200", "P_3", "P_1", "ndcg_cut_3"], perquery=True)

CPU times: user 429 ms, sys: 798 µs, total: 430 ms
Wall time: 428 ms


/data3/muntean/DRhard/DRhard/lib/python3.8/site-packages/pyterrier/pipelines.py:108: UserWarning: 306 topic(s) not found in qrels. Scores for these topics are given as NaN and should not contribute to averages.
  warn(f'{backfill_count} topic(s) not found in qrels. Scores for these topics are given as NaN and should not contribute to averages.')


In [67]:
res_per_query

,name,qid,measure,value
0,STAR,31_1,map,0.226870
1,STAR,31_1,recip_rank,1.000000
2,STAR,31_1,P_1,1.000000
3,STAR,31_1,P_3,1.000000
4,STAR,31_1,recall_200,0.438202
...,...,...,...,...
2863,STAR,80_9,recip_rank,NaN
2864,STAR,80_9,recall_200,NaN
2865,STAR,80_9,P_3,NaN
2866,STAR,80_9,P_1,NaN


# Explore the dictionaries

In [68]:
# distance dicts

# cache_radius_dict = dict() # between first utterance (qi) and last retrieved doc from the big index
# query_distance_dict = dict() # distance between the first (qi) and the rest of utterances of the conversation (q)
# query_radius_dict = dict() # between current utterance (q) and last retrieved doc from the small index
# diff_distance_dict = dict() # rbcapuccio = rqi - d(q, qi)

In [69]:
with open('../data/star-ranking/cache-radius-star-L2-ranking-top1000-cache-top'+str(topk)+'.pickle', 'wb') as handle:
    pickle.dump(cache_radius_dict, handle)

In [70]:
with open('../data/star-ranking/query-dist-star-L2-ranking-top1000-cache-top'+str(topk)+'.pickle', 'wb') as handle:
    pickle.dump(query_distance_dict, handle)

In [71]:
with open('../data/star-ranking/query-radius-star-L2-ranking-top1000-cache-top'+str(topk)+'.pickle', 'wb') as handle:
    pickle.dump(query_radius_dict, handle)

In [72]:
with open('../data/star-ranking/rbcapuccio-star-L2-ranking-top1000-cache-top'+str(topk)+'.pickle', 'wb') as handle:
    pickle.dump(diff_distance_dict, handle)

In [75]:
diff_distance_dict

{'31_2': 118418702.4013666,
 '31_3': 118418700.92902221,
 '31_4': 118418701.23348655,
 '31_5': 118418701.86292733,
 '31_6': 118418703.31605043,
 '31_7': 118418701.67394198,
 '31_8': 118418701.75246419,
 '31_9': 118418702.19444646,
 '32_2': 132400537.1040189,
 '32_3': 132400535.64936134,
 '32_4': 132400536.1166589,
 '32_5': 132400535.77234432,
 '32_6': 132400533.15507337,
 '32_7': 132400535.01224682,
 '32_8': 132400535.1097686,
 '32_9': 132400534.94515058,
 '32_10': 132400534.93758032,
 '32_11': 132400535.66604349,
 '33_2': 270514020.71085685,
 '33_3': 270514018.61838144,
 '33_4': 270514019.1643993,
 '33_5': 270514019.37582153,
 '33_6': 270514019.2843506,
 '33_7': 270514019.4926793,
 '33_8': 270514019.6423883,
 '33_9': 270514018.9256343,
 '33_10': 270514014.73496145,
 '34_2': 508512266.5087433,
 '34_3': 508512266.57521343,
 '34_4': 508512265.93703246,
 '34_5': 508512266.7227378,
 '34_6': 508512265.7272608,
 '34_7': 508512265.8864572,
 '34_8': 508512265.57458663,
 '34_9': 508512266.39690

In [76]:
query_radius_dict

{'31_2': 195019470.8074214,
 '31_3': 5294505.158112519,
 '31_4': 34332323.07639522,
 '31_5': 158067760.3044881,
 '31_6': 133634149.12005985,
 '31_7': 154450877.9551715,
 '31_8': 124908575.99352296,
 '31_9': 90299872.98001407,
 '32_2': 19571619.836761795,
 '32_3': 78685072.84893836,
 '32_4': 943334507.7418764,
 '32_5': 203165153.357555,
 '32_6': 121627545.24109851,
 '32_7': 1054655075.9210392,
 '32_8': 123378274.48142193,
 '32_9': 1042821039.6809728,
 '32_10': 162225596.79042092,
 '32_11': 185757571.63144034,
 '33_2': 784002045.4272158,
 '33_3': 568829041.0489368,
 '33_4': 846482965.4620757,
 '33_5': 562013462.6917564,
 '33_6': 841391844.596731,
 '33_7': 666755203.0610946,
 '33_8': 885052134.274929,
 '33_9': 405069650.8767906,
 '33_10': 286359599.70390195,
 '34_2': 955368685.870494,
 '34_3': 408387872.23913664,
 '34_4': 617255020.0267925,
 '34_5': 923005385.832172,
 '34_6': 672491395.0777481,
 '34_7': 1003828613.0707232,
 '34_8': 633061438.5702288,
 '34_9': 850734970.0540866,
 '37_2': 8